# Taller inicial de Julia

## Environments and the Package manger (Pkg)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("Parameters")
Pkg.add("Unitful")
Pkg.add("UnitfulAstro")
Pkg.add("Zygote")
Pkg.add("DataFrames")
Pkg.add("DifferentialEquations")

## Arrays

In [ ]:
# Arrays
a = [1.0, 2.0, 3.0]
@show a typeof(a)
@show typeof(a)<:Vector{Integer}
@show typeof(a)<:Vector{Float64};


In [ ]:
a[begin]==a[1]  # a[end]==a[3]

In [ ]:
@show sum(a)
b = 2a
@show c = a + b
@show c = a + b.*b
@show c = a .+ b'b;

In [ ]:
m = [a 2a 3a]
display(m)
display(m')

## Structs

In [ ]:
struct Evento{T<:Real,D<:Real}
    t::T
    x::D
    v::D
end

In [ ]:
a = Evento(0, 10., 1.)

In [ ]:
a = Evento(0, 10.) # show what happens

In [ ]:
Evento(x::T,y::T) where {T<:Real} = Evento(0., x, y)

In [ ]:
Evento(10., 1.)

In [ ]:
methods(Evento)

In [ ]:
Evento(; t=9, x=7, v) = Evento(t, x, v)

In [ ]:
a = Evento(t=0, x=1, v=1)

In [ ]:
b = Evento(t=98)

In [ ]:
b = Evento(v=54)

## Parameters package

In [ ]:
using Parameters
@with_kw struct Event{T<:Real,D<:Real}
    t::T = 0.0
    x::D = 5.0
    v::D
end

In [ ]:
methods(Event)

In [ ]:
b = Event(v=3) # then Event(v=3.0)

## Unitful Quantities

In [ ]:
using Unitful, UnitfulAstro
x = rand(3)u"kpc"
v = rand(3)u"km/s"
t = 10^6*u"s"
@show x v t

In [ ]:
a = v/t

In [ ]:
uconvert.(u"km/s/Myr",a)

In [ ]:
u"G"

In [ ]:
G = uconvert(u"kpc*Msun^-1*(km/s)^2",u"G")

## Automatic Differentiation

In [ ]:
using Zygote
f(x,y) = exp(x/y)*cos(3x+2y)

In [ ]:
∇f(x,y) = gradient((x,y)->f(x,y), x,y)

In [ ]:
∇f(1.,2.)

In [ ]:
∇f(x) = gradient(x->f(x...), x)

In [ ]:
methods(∇f)

In [ ]:
@show ∇f([1.,2.])[1]  ∇f(1.,2.)

In [ ]:
f(x,y) = exp(x)*exp(7y)
Df(x,y) = [1,7]*f(x,y)
∇f(x,y) = gradient((x,y)->f(x,y), x,y)
df(x,y) = gradient(f, x,y)

In [ ]:
x = [5.,-7.]
@show Df(x...) ∇f(x...) df(x...)
@show Df(x...) .== ∇f(x...);
@show Df(x...) .== df(x...);

## Data Frames

In [ ]:
using DataFrames
n=20
data = DataFrame(id=collect(1:n), x=rand(n), y=rand(n), z=rand(n))

In [ ]:
deleteat!(data, [3,5,7])

## Galactic Dynamics 

In [ ]:
using GalacticDynamics

In [ ]:
pot = Plummer(34u"Msun", 10u"pc")
pot₂ = NFW(10^3*u"Msun", 5u"kpc")
@show pot pot₂ pot+pot₂

In [ ]:
x=[5.0, 1.0, -2.0]u"pc"
@show potential(pot,x)
@show acceleration(pot,x)
@show potential(pot₂,x)
@show acceleration(pot₂,x)
@show potential(pot+pot₂,x)
@show acceleration(pot+pot₂,x);

### DifferentialEquations.jl https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/

In [ ]:
using DifferentialEquations
using Test

function example_orbit()
    m_gal = 2.325e7*u"Msun"
    m =1018.0*m_gal  # Msun
    a = 2.562*u"kpc"     # kpc
    Λ = 200.0*u"kpc"    # kpc
    γ = 2.0
    pot = AllenSantillanHalo(m, a, Λ, γ)
    for i in range(1,20)
        w₀ = 50*rand(6)
        x₀ = w₀[1:3]u"kpc"
        v₀ = w₀[4:6]u"km/s"
        t_range = (0.0,100.0).*𝕦.t
        sol = evolve(pot, x₀, v₀, t_range)
        sol₂ = evolve(pot, x₀, v₀, t_range; options=SolverConfig(solver=Tsit5(), reltol=5.0e-12))
        @show @test sol.x[end] ≈ sol₂.x[end] rtol=5.0e-8
        @show @test sol.v[end] ≈ sol₂.v[end] rtol=5.0e-8
    end
end

In [ ]:
example_orbit()